## Ques1

In [181]:
import numpy as np
import pandas as pd

In [182]:
df = pd.read_csv('weather.csv')
df

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,78,False,yes
3,rain,70,96,False,yes
4,rain,68,80,False,yes
5,rain,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rain,75,80,False,yes


In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Outlook      14 non-null     object
 1   Temperature  14 non-null     int64 
 2   Humidity     14 non-null     int64 
 3   Windy        14 non-null     bool  
 4   Play         14 non-null     object
dtypes: bool(1), int64(2), object(2)
memory usage: 590.0+ bytes


In [184]:
def find_entropy(df):
    entropy=0
    Class=df.keys()[-1]
    outcomes=df[Class].unique()
    for outcome in outcomes:
        # print(df[Class][df[Class]==outcome])
        prob=len(df[Class][df[Class]==outcome])/len(df)
        entropy+= -(prob*np.log2(prob))
    return entropy

In [185]:
find_entropy(df)

0.9402859586706311

In [186]:
def find_average_information_entropy(df,attribute):
    Class=df.keys()[-1]
    outcomes=df[Class].unique()
    unique_values=df[attribute].unique()
    average_info_entropy=0
    for value in unique_values:
        entropy_subsample=0
        for value1 in outcomes:
            num=len(df[attribute][df[attribute]==value][df[Class]==value1])
            den=len(df[attribute][df[attribute]==value])
            prob=num/den
            entropy_subsample+= -(prob*np.log2(prob+1e-10))
        weight=den/len(df)
        average_info_entropy+= weight*entropy_subsample
    return average_info_entropy

In [187]:
find_average_information_entropy(df,'Outlook')

0.6935361386488728

In [188]:
def split_info(df,attribute):
    Class=df.keys()[-1]
    outcomes=df[Class].unique()
    unique_values=df[attribute].unique()
    # print(unique_values)
    split_info_subsample=0
    for value in unique_values:
        num=len(df[attribute][df[attribute]==value])
        den=len(df)
        prob = num/den
        split_info_subsample+= -(prob*np.log2(prob+1e-10))
    return split_info_subsample

In [189]:
split_info(df,'Outlook')

1.577406282419537

In [190]:
def find_winner(df):
    GR=[]
    for key in df.keys()[:-1]:
        GR.append((find_entropy(df)-find_average_information_entropy(df,key))/split_info(df,key))
    return df.keys()[:-1][np.argmax(GR)]

In [191]:
find_winner(df)

'Temperature'

### (a) Encode the Temperature and Humidity as binary column wit values True/False using a threshold being decided by Gain Ratio.

In [192]:
def cont_to_nominal(df,attributes,result):
    for attr in attributes:
        df2 = pd.DataFrame()
        df2[attr] = df[attr]
        df2[result] = df[result]
        df2 = df2.sort_values(by=[attr]).reset_index(drop=True)
        GR=[]
        for threshold in list(df2[attr]):
            entropy = find_entropy(df)
            df3 = df2.copy()
            for i in range(len(df2[attr])):
                if df3[attr][i] <= threshold:
                    df3[attr][i] = True
                else:
                    df3[attr][i] = False


            for key in df3.keys()[:-1]:
                GR.append((find_entropy(df3)-find_average_information_entropy(df3,key))/split_info(df3,key))

        best_thershold_idx = GR.index(max(GR))
        # print(best_thershold_idx, df2['temp'][best_thershold_idx])
        for i in range(len(df2[attr])):
                if df[attr][i] <= df2[attr][best_thershold_idx]:
                    df[attr][i] = True
                else:
                    df[attr][i] = False

In [193]:
cont_to_nominal(df,['Temperature','Humidity'],'Play')

<ipython-input-192-5fd49ad0bca0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[attr][i] = True
C:\Python\Python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-192-5fd49ad0bca0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[attr][i] = False
<ipython-input-192-5fd49ad0bca0>:25: SettingWithCopyWarning: 
A value is tryin

In [194]:
df

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,False,False,False,no
1,sunny,True,False,True,no
2,overcast,True,False,False,yes
3,rain,True,False,False,yes
4,rain,True,False,False,yes
5,rain,True,False,True,no
6,overcast,True,True,True,yes
7,sunny,True,False,False,no
8,sunny,True,False,False,yes
9,rain,True,False,False,yes


### (b) Train a C4.5 Decision Tree Classifier on the transformed dataset.
### (c) Using Gain Ratio, find the best two attributes that decide the Play attribute.

In [196]:
def subtable(df,attribute,value):
    return df[df[attribute]==value].reset_index(drop=True)

In [197]:
def build_tree(df,tree=None):
    Class=df.keys()[-1]
    attribute=find_winner(df)
    if tree is None:
      tree={}
      tree[attribute]={}
    unique_value=df[attribute].unique()
    for value in unique_value:
        subsample=subtable(df,attribute,value)
        ClValue,counts=np.unique(subsample[Class],return_counts=True)
        if len(counts)==1:
            tree[attribute][value]=ClValue[0]
        else:
            if np.all(np.ones(len(counts)) == counts)!=True:
              tree[attribute][value]=build_tree(subsample)
            
    return tree


In [198]:
tree = build_tree(df)

In [199]:
tree 

{'Temperature': {False: 'no',
  True: {'Outlook': {'sunny': {'Windy': {}},
    'overcast': 'yes',
    'rain': {'Windy': {False: 'yes', True: 'no'}}}}}}

### From above we got Temperature and Outlook as top 2 atrribute for Play

### (d) Using 𝜒􀬶 test, check whether Play label depends upon Outlook feature or not at 95% confidence level.

In [210]:
outlook = df['Outlook'].unique()
play = df['Play'].unique()
df2 = pd.DataFrame(columns=play, index=outlook)
for i in outlook:
    val, count = np.unique(df[df['Outlook']==i]['Play'], return_counts = True)
    for j in range(len(val)):
        df2[val[j]][i] = count[j]

df2=df2.fillna(0)
df3

,no,yes
overcast,0,4
rain,2,3
sunny,3,2


In [205]:
observed = []
for i in df2.index:
    for j in df2.columns:
        observed.append(df2[j][i])
        # print(j,i)

observed

[0, 4, 2, 3, 3, 2]

In [206]:
row_total = df2.sum(axis=1)
col_total = df2.sum(axis=0)
n = row_total.sum()
print(row_total,col_total, n)

overcast    4
rain        5
sunny       5
dtype: int64 no     5
yes    9
dtype: int64 14


In [207]:
expected = []
for i in row_total:
    for j in col_total:
        p = (i/n)*(j/n)
        e = n*p
        expected.append(e)

expected

[1.4285714285714286,
 2.5714285714285716,
 1.7857142857142858,
 3.214285714285715,
 1.7857142857142858,
 3.214285714285715]

In [208]:
observed = np.array(observed)
expected = np.array(expected)

In [209]:
degree = (len(df2.index)-1)*(len(df2.columns)-1)
chi_square = np.sum((observed-expected)**2/expected)
print(degree,chi_square)


2 3.546666666666667


### P-value for 95% confidence for 2 degree freedom = 5.991 and our is 3.54 which is less so we accept null hypothesis

![alt text](https://www.statology.org/wp-content/uploads/2020/01/chi_square_table_small.jpg "Title")